In [ ]:
!pip install openai
!pip install spacy
!python -m spacy download en_core_web_lg

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
import time
import spacy
import openai
import string
import psutil
import logging
import numpy as np
import pandas as pd
import en_core_web_lg
from openai import OpenAI
from datetime import datetime

# Set up paths
from google.colab import drive
drive.mount('/content/drive/')
base_path = "/content/drive/MyDrive"

# Set up logging
log_file = f"{base_path}/performance_logs.log"
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def log_performance(function_name, start_time, end_time):
    """Log the performance metrics for a function."""
    execution_time = end_time - start_time
    process = psutil.Process()
    cpu_percent = process.cpu_percent()
    memory_info = process.memory_info()
    memory_usage = memory_info.rss / (1024 ** 2)  # Convert to MB

    log_message = (
        f"Function: {function_name}\n"
        f"Execution time: {execution_time:.2f} seconds\n"
        f"CPU usage: {cpu_percent:.2f}%\n"
        f"Memory usage: {memory_usage:.2f} MB\n"
        "-------------------------------"
    )
    logging.info(log_message)


# Load GloVe vectors
nlp_glove = spacy.load("en_core_web_lg")

# Set up OpenAI API key
# TODO: Remove this api key before committing
openai.api_key = "sk-proj-aXyKoTNJA8Cd4E7fP9s0T3BlbkFJKVQSd6folsAavHi3mQWi"
client = OpenAI(api_key=openai.api_key)

In [ ]:
from tqdm import tqdm

def convert_to_jsonl(df, output_file):
    """Convert a DataFrame to a JSONL file."""
    jsonl_list = [
        {"prompt": row["RSD Name"], "completion": row["Skill Statement"]}
        for _, row in df.iterrows()
    ]
    with open(output_file, "w") as jsonl_file:
        for jsonl_dict in jsonl_list:
            jsonl_file.write(
                f'{{"prompt": "{jsonl_dict["prompt"]}", "completion": "{jsonl_dict["completion"]}"}}\n'
            )

def get_embedding(text):
    """Calculate embeddings for a given text."""
    start_time = time.time()
    doc = nlp_glove(text)
    if len(doc) == 0:
        return np.zeros(300)  # Return zeros for empty texts
    end_time = time.time()
    log_performance("Get Embedding Perf:", start_time, end_time)
    return np.mean([word.vector for word in doc], axis=0)

def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors."""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def extract_skills_from_job_description(job_description, fine_tuned_model_id):
    """Extract skills from a job description using a fine-tuned model."""
    start_time = time.time()
    response = client.completions.create(
        model=fine_tuned_model_id,
        prompt=f"""Name all the skills present in the following job description in a single list.
                Response should have only the skills, no other information or words.
                Skills should be keywords, each being no more than 3 words.:
                This is the Job Description:
                {job_description}

                Skills:
                """,
        max_tokens=75,
        temperature=0.0,
    )
    extracted_skills = response.choices[0].text.strip()
    extracted_skills_set = set(
        [word.lstrip("-").strip() for word in extracted_skills.split("\n")]
    )
    end_time = time.time()
    log_performance("Extracted Skills perf for each query/job/course", start_time, end_time)
    return list(extracted_skills_set)

def compare_skills_with_glove(extracted_skills_list, taxn_source, similarity_threshold=0.65):
    """Match extracted skills with OSN skills using GloVe embeddings."""
    start_time = time.time()
    key_series = taxn_source["RSD Name"]
    skill_matches = []
    matched_skills_set = set()

    for extracted_skill in tqdm(extracted_skills_list):
        if extracted_skill.strip():
            extracted_embedding = get_embedding(extracted_skill)
            best_match = None
            best_similarity = 0.0

            for key_skill in key_series:
                key_embedding = get_embedding(key_skill)
                similarity = cosine_similarity(extracted_embedding, key_embedding)

                if (
                    similarity >= similarity_threshold
                    and key_skill not in matched_skills_set
                ):
                    best_similarity = similarity
                    best_match = key_skill
                    matched_skills_set.add(key_skill)

            if best_match:
                skill_matches.append(best_match)

    end_time = time.time()
    log_performance("compare_skills_with_glove Perf.", start_time, end_time)
    return skill_matches

import multiprocessing as mp
from functools import partial

def match_skills_for_job(job_desc, fine_tuned_model_id, taxn_source, similarity_threshold=0.65):
    """Match skills for a single job description."""
    extracted_skills = extract_skills_from_job_description(job_desc, fine_tuned_model_id)
    job_matches = compare_skills_with_glove(extracted_skills, taxn_source, similarity_threshold)
    return job_matches

def match_skills_for_job_df_parallel(jobs_df, fine_tuned_model_id, taxn_source, similarity_threshold=0.65, num_processes=None):
    """Match skills for each job in a DataFrame using parallel processing."""
    start_time = time.time()
    if num_processes is None:
        num_processes = mp.cpu_count()

    pool = mp.Pool(processes=num_processes)
    match_func = partial(match_skills_for_job, fine_tuned_model_id=fine_tuned_model_id, taxn_source=taxn_source, similarity_threshold=similarity_threshold)
    matched_skills_list = pool.map(match_func, [job_row["job_desc"] for _, job_row in jobs_df.iterrows()])
    pool.close()
    pool.join()

    matched_skills_df = pd.DataFrame(
        [{"Job Number": job_index + 1, "Matched Skills": job_matches}
         for job_index, job_matches in enumerate(matched_skills_list)],
        columns=["Job Number", "Matched Skills"]
    )
    end_time = time.time()
    log_performance("match_skills_for_job_df_parallel perf.", start_time, end_time)
    return matched_skills_df

def match_skills_for_job_df(jobs_df, fine_tuned_model_id, taxn_source, similarity_threshold=0.65):
    """Match skills for each job in a DataFrame."""
    start_time = time.time()
    matched_skills_list = []

    for job_index, job_row in tqdm(jobs_df.iterrows()):
        extracted_skills = extract_skills_from_job_description(
            job_row["job_desc"], fine_tuned_model_id
        )
        job_matches = compare_skills_with_glove(extracted_skills, taxn_source, similarity_threshold)
        job_data = {
            "Job Number": job_index + 1,
            "Matched Skills": job_matches,
        }
        matched_skills_list.append(job_data)

    matched_skills_df = pd.DataFrame(matched_skills_list)
    end_time = time.time()
    log_performance("match_skills_for_job_df perf.", start_time, end_time)
    return matched_skills_df

def find_common_skills(job_skills_df, min_matches=3):
    """Find common skills between jobs."""
    start_time = time.time()
    common_skills_pairs = []

    for job_index, job_row in tqdm(job_skills_df.iterrows()):
        current_job_number = job_row["Job Number"]
        current_job_skills = set(job_row["Matched Skills"])

        for other_job_index, other_job_row in job_skills_df.iloc[job_index + 1:].iterrows():
            other_job_number = other_job_row["Job Number"]
            other_job_skills = set(other_job_row["Matched Skills"])
            common_skills = current_job_skills.intersection(other_job_skills)
            num_common_skills = len(common_skills)

            if num_common_skills >= min_matches:
                common_skills_pairs.append(
                    (current_job_number, other_job_number, list(common_skills))
                )

    skills_common_df = pd.DataFrame(
        common_skills_pairs, columns=["Job Number 1", "Job Number 2", "Common Skills"]
    )
    end_time = time.time()
    log_performance("find_common_skills perf.", start_time, end_time)
    return skills_common_df

In [ ]:
import spacy
import torch

# Load GloVe vectors
nlp_glove = spacy.load("en_core_web_lg")

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the GloVe vectors to GPU if available
if device.type == "cuda":
    glove_vectors = nlp_glove.vocab.vectors.data
    glove_vectors_tensor = torch.tensor(glove_vectors, device=device)
    nlp_glove_gpu = spacy.vocab.Vocab(vectors=spacy.vocab.Vectors(data=glove_vectors_tensor))
    nlp_gpu = spacy.lang.en.English(vocab=nlp_glove_gpu)
else:
    nlp_gpu = nlp_glove
# Update the get_embedding function to use GPU
def get_embedding(text):
    """Calculate embeddings for a given text using GPU if available."""
    doc = nlp_glove(text)
    if len(doc) == 0:
        return torch.zeros(300, device=device)  # Return zeros for empty texts
    return torch.mean(
        torch.stack([torch.tensor(word.vector, device=device) for word in doc]), dim=0
    )

# Update the cosine_similarity function to use GPU
def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors on GPU if available."""
    vec1 = vec1.to(device)
    vec2 = vec2.to(device)
    return torch.dot(vec1, vec2) / (vec1.norm() * vec2.norm())

# Count common Skills
def count_common_skills(common_skills_df, job_skills_df):
    """Count the number of jobs that share a set of common skills."""
    start_time = time.time()
    count_skills = pd.DataFrame(columns=['Count', 'Jobs', 'Common Skills'])

    for index, row in tqdm(common_skills_df.iterrows()):
        common_skills_set = set(row['Common Skills'])
        count = 0
        job_numbers = []

        for job_index, job_row in job_skills_df.iterrows():
            matched_skills_set = set(job_row['Matched Skills'])

            if common_skills_set.issubset(matched_skills_set):
                count += 1
                job_numbers.append(job_row['Job Number'])

        count_skills = pd.concat([count_skills, pd.DataFrame({
            'Count': [count],
            'Jobs': [job_numbers],
            'Common Skills': [row['Common Skills']]
        })], ignore_index=True)
    end_time = time.time()
    log_performance("count_common_skills perf.", start_time, end_time)
    return count_skills

In [ ]:
# Load data
osn_comp_df = pd.read_csv('https://raw.githubusercontent.com/phanindra-max/LAiSER-datasets/master/osn_comp.csv')
osn_pub_df = pd.read_csv('https://raw.githubusercontent.com/phanindra-max/LAiSER-datasets/master/osn_public_rel.csv')
jobs_df = pd.read_csv('https://raw.githubusercontent.com/phanindra-max/LAiSER-datasets/master/jobs_df.csv')

# Convert OSN data to JSONL format
# convert_to_jsonl(osn_comp_df, "osn_comp.jsonl")
# convert_to_jsonl(osn_pub_df, "osn_pub.jsonl")

# Fine-tuned model IDs
fine_tuned_model_comp = "ft:davinci-002:personal::8IIFVUbf"

# Extract and match skills
job_skills_comp = match_skills_for_job_df_parallel(
    jobs_df, fine_tuned_model_comp, osn_comp_df, similarity_threshold=0.65
)
job_skills_pub = match_skills_for_job_df_parallel(
    jobs_df, fine_tuned_model_comp, osn_pub_df, similarity_threshold=0.65
)


In [ ]:
# Find common skills
common_skills_comp = find_common_skills(job_skills_comp)
common_skills_pub = find_common_skills(job_skills_pub)

# Count common skills
count_skills_comp = count_common_skills(common_skills_comp, job_skills_comp)
count_skills_pub = count_common_skills(common_skills_pub, job_skills_pub)

# Save results
job_skills_comp.to_csv(f"{base_path}/job_skills_comp.csv", index=False)
job_skills_pub.to_csv(f"{base_path}/job_skills_pub.csv", index=False)
count_skills_comp.to_csv(f"{base_path}/count_skills_comp.csv", index=False)
count_skills_pub.to_csv(f"{base_path}/count_skills_pub.csv", index=False)